In [1]:
import numpy as np

# Tarea

## Primer ejercicio

Hallar el area exacta de la región entre la parábola $y=x^3$ y el eje $x$ en el intervalo $[0, b]$. Usar las sumas de Riemann hacia la derecha de igual ancho.

Para esto, empezamos definiendo el número de intervalos $n$ que vamos a utilizar, y con esto, podemos conocer el ancho de cada _bin_ en el eje $x$ como $\Delta x = b/n$ (considerando que $a=0$). De esta forma, tenemos el set de puntos a evaluar $\{ x_i \}_{i=1}^n$

$$
x_i = i\frac{b}{n}; \qquad i = 1, 2, \cdots, n
$$

a partir del cual podemos definir la suma de Riemann

$$
\begin{align}
    S_n &= \sum_{i=1}^n f(x_i) \Delta x 
     = \sum_{i = 1}^n \left( i \frac{b}{n} \right)^3 \Delta x \\ 
     &=  \frac{b^3}{n^3} \Delta x \sum_{i=1}^n i^3 \\
     &= \frac{b^4}{n^4} \sum_{i=1}^n i^3
\end{align}
$$

Ahora, debemos resolver la suma interna, y para ello debemos usar la identidad

$$
\sum_{i=1}^n i^3 = \left( \frac{n(n+1)}{2} \right)^2
$$

y operando con lo anterior

$$
\begin{align}
    S_n &= \frac{b^4}{n^4} \left( \frac{n(n+1)}{2} \right)^2 \\ 
    &= \frac{b^4}{4} \frac{(n+1)^2}{n^2}
\end{align}
$$

y por último, consideramos el caso cuando $n\to\infty$, en donde tenemos que hallar

$$
\lim\limits{n\to\infty} \frac{(n+1)^2}{n^2} = 1
$$

donde tanto denominador como numerador tienen el mismo grado $n^2$, por ende, en el límite ambos se cancelan, llegando a que

$$
\boxed{
    S = \lim\limits_{n\to\infty} S_n = \frac{b^4}{4}
}
$$

Exactamente el resultado que esperamos.

# Segundo ejercicio

Para la función del cohete, hacer una tabla del comportamiento del método de Riemann, trapezio y Simpson variando para diferentes $n$

## Definición de las integrales


Para ello, empezamos primero por crear las funciones de las que obtenemos las integrales. Para ello, recordemos cómo se definen cada una de ellas. 

Considerando que queremos aproximar la integral 

$$
\int_a^b f(x) dx
$$

con $n$ las divisiones de la partición, y con un ancho de bin de $\Delta x = \frac{b - a}{n}$, se definen

### Riemann a la derecha

$$
S_n = \sum_{i=1}^{n} f(a + i \Delta x) \Delta x
$$

### Método del trapezio

$$
T_n = \frac{\Delta x}{2} \left( f(a) + 2 \sum_{i=1}^{n-1} f(a + i \Delta x) + f(b) \right)
$$


### Método de Simpson 1/3

Es importante considerar que $n$ debe ser **par**

$$
S_n = \frac{\Delta x}{3} \left( f(a) + f(b) + 4 \sum_{i = 1, 3, 5, \dots}^{n-1} f(a + i\Delta x) + 2 \sum_{i = 2, 4, 6, \dots}^{n-2} f(a + i \Delta x)\right)
$$


In [6]:
def riemann_right(f, a, b, n, rv):
    
    x = np.linspace(a, b, n + 1)
    dx = (b - a) / n

    res = dx * np.sum(f(x[1:]))
    
    return (res, np.abs(rv - res), 1 - np.abs( res / rv ))


def trapezoidal(f, a, b, n, rv):
    
    x = np.linspace(a, b, n + 1)
    dx = (b - a) / n
    
    y = f(x)
    
    res = dx * 0.5 * (y[0] + 2.0*np.sum(y[1:-1]) + y[-1])
    
    return (res, np.abs(rv - res), 1 - np.abs( res / rv ))


def simpson_13(f, a, b, n, rv):

    if n%2 != 0:
        # Esto me facilita hacer la tabla
        return ("---", "---", "---")
        
    x = np.linspace(a, b, n + 1)
    dx = (b - a) / n

    y = f(x)
    
    res = (dx/3) * (
        y[0]
        + 4 * np.sum(y[1:-1:2])
        + 2 * np.sum(y[2:-2:2])
        + y[-1]
    )

    return (res, np.abs(rv - res), 1 - np.abs( res / rv ))

## Otras funciones de ayuda

En este caso, como queremos una tabla, voy a crear una función que cree dicha tabla

*_Nota:_* _para que quede decentemente bonita, tuve que hacerla en HTML_

In [14]:
from scipy.integrate import quad

def createTable(f, a, b, ns, metFuncs = [riemann_right, trapezoidal, simpson_13]):
    real_value, _ = quad(f, a, b)   # valor "real" dado por librería

    for n in ns:
        print("<tr>")

        print(f"\t<td>{n}</td>", end="")
        for dv in metFuncs:

            val, err_abs, err_rel = dv(f, a, b, n, real_value)
            if val == "---":
                print(f"<td>{val}</td><td>{err_abs}</td><td>{err_rel}</td>", end="")
            else:
                print(f"<td>{val:.3g}</td><td>{err_abs:.3g}</td><td>{err_rel:.3g}</td>", end="")

        print("\n</tr>")
        


## Para la función del cohete

En este caso, tenemos que programar la función del cohete vista en clase

$$
x = \int_{8}^{30} \left( 2000 \ln\left[ \frac{140000}{140000 - 2100t} \right] - 9.8t \right) dt
$$ 

y a partir de ella, crear la tabla

In [19]:

def cohete_func(t):
    return 2000 * np.log( 140000 / (140000 - 2100*t) ) - 9.8*t

# Con esto creo la tabla facilmente
# createTable(cohete_func, 8, 30, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

<table border="2">
  <tr>
    <th rowspan="2">n</th><th colspan="3">Riemann</th><th colspan="3">Trapezoid</th><th colspan="3">Simpson 1/3</th>
  </tr>
  <tr>
    <td>x</td><td>err. abs</td><td>err. rel</td><td>x</td><td>err. abs</td><td>err. rel</td><td>x</td><td>err. abs</td><td>err. rel</td>
  </tr>
  <tr>
	<td>1</td><td>1.98e+04</td><td>8.78e+03</td><td>-0.793</td><td>1.19e+04</td><td>807</td><td>-0.073</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>2</td><td>1.53e+04</td><td>4.19e+03</td><td>-0.379</td><td>1.13e+04</td><td>205</td><td>-0.0185</td><td>1.11e+04</td><td>4.38</td><td>-0.000396</td>
</tr>
<tr>
	<td>3</td><td>1.38e+04</td><td>2.75e+03</td><td>-0.248</td><td>1.12e+04</td><td>91.4</td><td>-0.00827</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>4</td><td>1.31e+04</td><td>2.04e+03</td><td>-0.185</td><td>1.11e+04</td><td>51.5</td><td>-0.00465</td><td>1.11e+04</td><td>0.301</td><td>-2.72e-05</td>
</tr>
<tr>
	<td>5</td><td>1.27e+04</td><td>1.63e+03</td><td>-0.147</td><td>1.11e+04</td><td>33</td><td>-0.00298</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>6</td><td>1.24e+04</td><td>1.35e+03</td><td>-0.122</td><td>1.11e+04</td><td>22.9</td><td>-0.00207</td><td>1.11e+04</td><td>0.0606</td><td>-5.48e-06</td>
</tr>
<tr>
	<td>7</td><td>1.22e+04</td><td>1.16e+03</td><td>-0.104</td><td>1.11e+04</td><td>16.8</td><td>-0.00152</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>8</td><td>1.21e+04</td><td>1.01e+03</td><td>-0.0912</td><td>1.11e+04</td><td>12.9</td><td>-0.00116</td><td>1.11e+04</td><td>0.0193</td><td>-1.75e-06</td>
</tr>
<tr>
	<td>9</td><td>1.2e+04</td><td>896</td><td>-0.081</td><td>1.11e+04</td><td>10.2</td><td>-0.000921</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>10</td><td>1.19e+04</td><td>805</td><td>-0.0728</td><td>1.11e+04</td><td>8.25</td><td>-0.000746</td><td>1.11e+04</td><td>0.00793</td><td>-7.17e-07</td>
</tr>


## Función que yo elija 

En este caso, voy a programar una gaussiana estandar, conociendo que su integral en el dominio completo debe ser 1. 

$$
G(x) = \frac{1}{\sqrt{2\pi}} e^{- \frac{x^2}{2}}
$$

teniendo entonces que integrar

$$
\int_{-100}^{100} G(x) dx \approx 1.0
$$

In [28]:
def standard_gaussian(x):
    return (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * x**2)

# createTable(standard_gaussian, -5, 5, [n for n in range(1, 11)])

<table border="2">
  <tr>
    <th rowspan="2">n</th><th colspan="3">Riemann</th><th colspan="3">Trapezoid</th><th colspan="3">Simpson 1/3</th>
  </tr>
  <tr>
    <td>x</td><td>err. abs</td><td>err. rel</td><td>x</td><td>err. abs</td><td>err. rel</td><td>x</td><td>err. abs</td><td>err. rel</td>
  </tr>
 <tr>
	<td>1</td><td>1.49e-05</td><td>1</td><td>1</td><td>1.49e-05</td><td>1</td><td>1</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>2</td><td>1.99</td><td>0.995</td><td>-0.995</td><td>1.99</td><td>0.995</td><td>-0.995</td><td>2.66</td><td>1.66</td><td>-1.66</td>
</tr>
<tr>
	<td>3</td><td>0.663</td><td>0.337</td><td>0.337</td><td>0.663</td><td>0.337</td><td>0.337</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>4</td><td>1.09</td><td>0.085</td><td>-0.085</td><td>1.09</td><td>0.085</td><td>-0.085</td><td>0.782</td><td>0.218</td><td>0.218</td>
</tr>
<tr>
	<td>5</td><td>0.986</td><td>0.0144</td><td>0.0144</td><td>0.986</td><td>0.0144</td><td>0.0144</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>6</td><td>1</td><td>0.00164</td><td>-0.00164</td><td>1</td><td>0.00164</td><td>-0.00164</td><td>1.11</td><td>0.114</td><td>-0.114</td>
</tr>
<tr>
	<td>7</td><td>1</td><td>0.000128</td><td>0.000128</td><td>1</td><td>0.000128</td><td>0.000128</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>8</td><td>1</td><td>5.24e-06</td><td>-5.24e-06</td><td>1</td><td>5.24e-06</td><td>-5.24e-06</td><td>0.972</td><td>0.0283</td><td>0.0283</td>
</tr>
<tr>
	<td>9</td><td>1</td><td>1.31e-06</td><td>1.31e-06</td><td>1</td><td>1.31e-06</td><td>1.31e-06</td><td>---</td><td>---</td><td>---</td>
</tr>
<tr>
	<td>10</td><td>1</td><td>9.2e-07</td><td>9.2e-07</td><td>1</td><td>9.2e-07</td><td>9.2e-07</td><td>1</td><td>0.00479</td><td>-0.00479</td>
</tr>

Aquí vemos algo rarísimo: Simpson converge más lento que los otros métodos. Esto es devido a que Simpson se aprovecha del comportamiento polinómico de las funciones, mientras que una distribución normal, sobre todo en los extremos, no se comporta en lo absoluto parecido a un polinomio.

# Regla de Simpson compuesta

Sin querer ya la utilicé. Básicamente, lo que esta hace es "generalizar" la regla de Simpson vista originalmente (con una única partición) a una de $n$ divisiones. Es básicamente hacer lo que se hizo para el caso de $n=1$ pero, esta vez, "fiteando" polinomios cuadráticos en cada uno de los intervalos. 
$$
S_n = \frac{\Delta x}{3} \left( f(a) + f(b) + 4 \sum_{i = 1, 3, 5, \dots}^{n-1} f(a + i\Delta x) + 2 \sum_{i = 2, 4, 6, \dots}^{n-2} f(a + i \Delta x)\right)
$$


# Regla de Simpson de 3/8

Es en esencia lo mismo a lo anterior, pero esta vez fiteando polinomios cúbicos en un intervalo de 3 puntos.

Su formula general es, dado un $\Delta x = (b-a)/3$

$$
    S = h\frac{3}{8} \left( f(a) + 3f(a + \Delta x) + 3 f(a + 2 \Delta x) + f(b) \right)
$$

Existe igualmente una versión compuesta de esta regla.

El resultado de aplicar esta regla a la función que elejimos es

$$
x = \int_{-5}^{5} G(x) dx 
$$

In [30]:
def simpson_38(f, a, b):
    h = (b - a) / 3
    
    x0 = a
    x1 = a + h
    x2 = a + 2*h
    x3 = b
    
    return (3*h/8) * (
        f(x0)
        + 3*f(x1)
        + 3*f(x2)
        + f(x3)
    )

print(f"Resultado de aplicar Simpson 3/8 en una gaussiana: {simpson_38(standard_gaussian, -5, 5)}")

Resultado de aplicar Simpson 3/8 en una gaussiana: 0.7460822577444021
